In [8]:
from __future__ import print_function

import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from tqdm import tqdm


from log import Logger
from data import gDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import autoencoder



In [3]:
if not os.path.exists('./gal_img'):
    os.mkdir('./gal_img')

In [4]:


def to_img(x):
    x = x.view(x.size(0), 1, 96, 96)
    return x

num_epochs =1000
batch_size = 64
learning_rate = 1e-4


def plot_sample_img(img, name):
    img = img.view(1, 96, 96)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

In [5]:
dataset= gDataset()
dataloader= DataLoader(dataset=dataset, batch_size=64,shuffle=True)

In [6]:
model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [7]:
for epoch in range(num_epochs):
    for data in dataloader:
        img = data
        
        
        #
        img.type(torch.float32)

        img = img.view(img.size(0), -1)
        img = Variable(img).cuda().type('torch.FloatTensor')
        

        # ===================forward=====================

        output = model(img.cuda())

        loss = criterion(output.cuda(), img.cuda())
        MSE_loss = nn.MSELoss()(output.cuda(), img.cuda())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data, MSE_loss.data))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        save_image(x, './gal_img/x_{}.png'.format(epoch))
        save_image(x_hat, './gal_img/x_hat_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/1000], loss:0.4724, MSE_loss:0.1395
epoch [2/1000], loss:0.1702, MSE_loss:0.0223
epoch [3/1000], loss:0.0648, MSE_loss:0.0030


KeyboardInterrupt: 